<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/single%20task/source%20code%20summarization/csharp/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the library and download the pretrained models

In [1]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5==0.6.4

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

!wget "https://www.dropbox.com/sh/kjoqdpj7e16dny9/AADdvjWVFckCgNQN-AqMKhiDa?dl=1" -O vocabulary.zip
!unzip vocabulary.zip
!rm vocabulary.zip
!wget "https://www.dropbox.com/sh/wi1xsssqtu130yn/AADZ2dJXMQbtSwJFv-twDDMUa?dl=1" -O csharp.zip
!unzip csharp.zip
!rm csharp.zip

Installing dependencies...
     |████████████████████████████████| 163kB 5.9MB/s 
     |████████████████████████████████| 348kB 33.1MB/s 
     |████████████████████████████████| 1.1MB 48.1MB/s 
     |████████████████████████████████| 1.3MB 39.8MB/s 
     |████████████████████████████████| 3.7MB 28.3MB/s 
     |████████████████████████████████| 2.6MB 44.5MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 2.9MB 42.1MB/s 
     |████████████████████████████████| 890kB 45.0MB/s 
ERROR: transformers 3.5.0 has requirement sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.
build configs
INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 
--2020-11-12 10:37:13--  https://www.dropbox.com/sh/kjoqdpj7e16dny9/AADdvjWVFckCgNQN-AqMKhiDa?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com 

## Set sentencepiece model

In [2]:
from t5.data.sentencepiece_vocabulary import SentencePieceVocabulary

vocab_model_path = 'code_spm_unigram_40M.model'
vocab = SentencePieceVocabulary(vocab_model_path, extra_ids=100)

print("Vocab has a size of %d\n" % vocab.vocab_size)

Vocab has a size of 32100



## Set the preprocessors and the task registry for the t5 model

In [3]:
def csharp_sourceSum_dataset_fn(split, shuffle_files=False):
    del shuffle_files

    ds = tf.data.TextLineDataset(csharp_path[split])
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""], field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    ds = ds.map(lambda *ex: dict(zip(["code", "docstring"], ex)))
    return ds


def csharp_preprocessor(ds):
    def normalize_text(text):
        return text

    def to_inputs_and_targets(ex):
        return {
            "inputs": tf.strings.join(["source code summarization csharp: ", normalize_text(ex["code"])]),
            "targets": normalize_text(ex["docstring"])
        }

    return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)


t5.data.TaskRegistry.remove('sourcesum_csharp_code')
t5.data.TaskRegistry.add(
    "sourcesum_csharp_code",
    dataset_fn=csharp_sourceSum_dataset_fn,
    output_features={
        "inputs": t5.data.utils.Feature(vocabulary=vocab),
        "targets": t5.data.utils.Feature(vocabulary=vocab),
    },
    splits=["train", "validation"],
    text_preprocessor=[csharp_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.bleu, t5.evaluation.metrics.accuracy, t5.evaluation.metrics.rouge],
)

## Set t5 base model

In [4]:
MODEL_DIR = "base"
model_parallelism = 1
train_batch_size = 256

tf.io.gfile.makedirs(MODEL_DIR)
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=None,
    tpu_topology=None,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    mesh_shape="model:1,batch:1",
    mesh_devices=["GPU:0"],
    learning_rate_schedule=0.003,
    save_checkpoints_steps=500,
    keep_checkpoint_max=None,
    iterations_per_loop=100,
)

## Code Documentation Summarization

### Give the code for summarization

In [5]:
code = "public static DateTime ParseUnixDateTime(double unixTime)\n    {\n        var dt= new DateTime(1970, 1, 1, 0, 0, 0, 0, System.DateTimeKind.Utc);\n        dt= dt.AddSeconds(unixTimeStamp).ToLocalTime();\n        return dt;\n    }" #@param {type:"raw"}


### Parsing and Tokenization

In [6]:
!pip install antlr4-python3-runtime==4.5.2
import antlr4

!wget https://www.dropbox.com/s/o87gk1jxf8645eu/CSharp4Lexer.py?dl=1 -O CSharp4Lexer.py
from CSharp4Lexer import CSharp4Lexer

     |████████████████████████████████| 112kB 4.2MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.5.2-cp36-none-any.whl size=165556 sha256=87ba221269d2c47f3a54c7f79a211bbaaa2b6ac921ecbe95101eb87b909737cd
  Stored in directory: /root/.cache/pip/wheels/e8/8f/2d/020b9595d9afea75db47d851fea1812250bae782bba902c72b
Successfully built antlr4-python3-runtime
--2020-11-12 10:37:37--  https://www.dropbox.com/s/o87gk1jxf8645eu/CSharp4Lexer.py?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/o87gk1jxf8645eu/CSharp4Lexer.py [following]
--2020-11-12 10:37:37--  https://www.dropbox.com/s/dl/o87gk1jxf8645eu/CSharp4Lexer.py
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc723a5f78d412e6cfb8dd763ff0.dl

In [7]:
def csTokenizer(line):
    l = line.replace('\\n', '\n')
    parsedVersion = []
    stream = antlr4.InputStream(l)
    lexer = CSharp4Lexer(stream)
    toks = antlr4.CommonTokenStream(lexer)
    toks.fetch(500)

    identifiers = {}
    identCount = 0
    for token in toks.tokens:
      if token.type == 109:
        parsedVersion.append("CODE_INTEGER")
      elif token.type == 111:
        parsedVersion.append("CODE_REAL")
      elif token.type == 112:
        parsedVersion.append("CODE_CHAR")
      elif token.type == 113:
        parsedVersion.append("CODE_STRING")
      elif token.type == 9 or token.type == 7 or token.type == 6 or token.type == 4 or token.type == 8 or token.type == 5: # whitespace and comments and newline
        pass
      else:
        parsedVersion.append(str(token.text))

    parsedVersion.remove('<EOF>')
    return ' '.join(parsedVersion)

tokenized_code = csTokenizer(code)
print("code after tokenization: " + tokenized_code)

code after tokenization: public static DateTime ParseUnixDateTime ( double unixTime ) { var dt = new DateTime ( CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , System . DateTimeKind . Utc ) ; dt = dt . AddSeconds ( unixTimeStamp ) . ToLocalTime ( ) ; return dt ; }


### Record the code for summarization with the prefix to a txt file

In [8]:
codes = [tokenized_code]

inputs_path = 'input.txt'
with tf.io.gfile.GFile(inputs_path, "w") as f:
  for c in codes:
    f.write("source code summarization csharp: %s\n" % c)

predict_outputs_path = 'MtfModel-output.txt'


### Running the model with the best checkpoint to summarize the given code

In [9]:
model.batch_size = 8  
model.predict(
    input_file="input.txt",
    output_file=predict_outputs_path,
    checkpoint_steps=500,
    beam_size=4,
    vocabulary=vocab, 
    # Select the most probable output token at each step.
    temperature=0,
)

INFO:tensorflow:Using config: {'_model_dir': 'base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_se

### Code Summarization Result

In [10]:
prediction_file = "MtfModel-output.txt-500"
print("\nPredictions using checkpoint 500:\n" )
with tf.io.gfile.GFile(prediction_file) as f:
  for c, d in zip(codes, f):
    if c:
      print("Code for prediction: " + c + '\n')
      print("Generated Summarization: " + d)



Predictions using checkpoint 500:

Code for prediction: public static DateTime ParseUnixDateTime ( double unixTime ) { var dt = new DateTime ( CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , CODE_INTEGER , System . DateTimeKind . Utc ) ; dt = dt . AddSeconds ( unixTimeStamp ) . ToLocalTime ( ) ; return dt ; }

Generated Summarization: b'How to convert DateTime to DateTime'

